In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import json
from pprint import pprint

from src.supabase_client import get_user
from tests.common import run_new_training_week_process_wrapped, run_mid_week_update_process_wrapped


# Test with Jamie

In [5]:
mock_user = get_user(os.environ["JAMIES_ATHLETE_ID"])
response = run_new_training_week_process_wrapped(mock_user)
print(response)

athlete_id=98390356 token still valid until 2024-08-30 23:49:42+00:00
athlete_id=98390356 token still valid until 2024-08-30 23:49:42+00:00
TrainingWeekWithCoaching(typical_week_training_review=Your current training schedule effectively incorporates long runs on Saturdays, which aligns well with your stated preference. Wednesdays appear to be ideal for your workouts as you log higher mileage with a pace of 9m 20s per mile, suggesting potential for speed work on this day. Your lighter runs and potential rest days could be on Sundays, allowing for recovery post long runs., weekly_mileage_target=For the upcoming week, let's aim for a target weekly mileage of around 47 miles. Your longest run should range between 16 to 17 miles, ensuring a gradual increase without overexertion. Over recent weeks, you've successfully handled increasing mileage and can now focus on building endurance for your next marathon. Be attentive to your body's feedback, especially since you've had a consistent traini

In [4]:
mock_user = get_user(os.environ["JAMIES_ATHLETE_ID"])
response = run_mid_week_update_process_wrapped(mock_user)
print(response)

athlete_id=98390356 token still valid until 2024-08-30 23:49:42+00:00
athlete_id=98390356 token still valid until 2024-08-30 23:49:42+00:00
TrainingWeekWithPlanning(planning=Planning(weekly_mileage_target=47.0, long_run_distance=16.0, remaining_weekly_mileage=47.0 - 10.04 = 36.96, remaining_weekly_mileage_planning=To achieve a total of 36.96 miles remaining for this week, while keeping the long run at 16.0 miles, we'll adjust the rest of the week's schedule: 

- Wednesday (Speed workout): Keep the planned 7.0 miles. 
- Thursday (Easy run): We'll adjust this to 4.0 miles for better mileage distribution. 
- Friday (Moderate run): Reduce from 6.0 to 5.0 miles to adjust total mileage.
- Saturday (Long run): Keep as planned at 16.0 miles. 
- Sunday (Easy run): Change this from 4.0 miles to 4.96 miles to reach the exact mileage target. 

Calculating total adjusted mileage: 7.0 (Wed) + 4.0 (Thu) + 5.0 (Fri) + 16.0 (Sat) + 4.96 (Sun) = 36.96 miles, precisely matching the required mileage.), tr

In [6]:
from src.activities import get_activity_summaries
from src.auth_manager import get_strava_client
from src.constants import COACH_ROLE
from src.supabase_client import get_training_week_with_coaching
from src.training_week_update import get_updated_training_week
from src.types.mid_week_analysis import MidWeekAnalysis


user = get_user(os.environ["JAMIES_ATHLETE_ID"])

sysmsg_base = f"{COACH_ROLE}\nYour client has included the following preferences: {user.preferences}\n"
strava_client = get_strava_client(user.athlete_id)

training_week_with_coaching = get_training_week_with_coaching(user.athlete_id)
current_weeks_activity_summaries = get_activity_summaries(
    strava_client, num_weeks=1
)
mid_week_analysis = MidWeekAnalysis(
    activities=current_weeks_activity_summaries,
    training_week=training_week_with_coaching.training_week,
)
training_week_update_with_planning = get_updated_training_week(
    sysmsg_base=sysmsg_base, mid_week_analysis=mid_week_analysis
)

athlete_id=98390356 token still valid until 2024-08-30 23:49:42+00:00


In [15]:
mid_week_analysis.activities[0].pace_minutes_per_mile
# Training week accuracy

nan

# Training week accuracy

In [24]:
path = 'tests/artifacts/new_training_week_errors'
files = os.listdir(path)

for file in sorted(files):
    print(f"Loading {file}")
    with open(os.path.join(path, file), "r") as f:
        results = json.load(f)
    print(sum(result['error'] for result in results) / len(results))

Loading 2024-08-28_16-47-43.json
1.97
Loading 2024-08-28_17-27-11.json
1.95
Loading 2024-08-28_17-41-17.json
1.95
Loading 2024-08-29_17-37-07.json
0.75
Loading 2024-08-29_17-45-21.json
0.95
Loading 2024-08-29_17-54-20.json
0.425
Loading 2024-08-29_18-09-32.json
0.475
Loading 2024-08-30_17-57-20.json
0.85
Loading 2024-08-30_18-17-10.json
0.7885
Loading 2024-08-30_18-28-10.json
0.6125
Loading 2024-08-30_19-16-03.json
0.34550000000000003
Loading 2024-08-30_19-21-24.json
0.2
Loading 2024-08-30_19-34-15.json
0.6
Loading 2024-08-30_19-42-10.json
0.4
Loading 2024-08-30_19-48-21.json
0.4


In [25]:
for result in results:
    print(result['user']['email'])
    print("target:", result['recommended_weekly_mileage'])
    print("actual:", result['generated_weekly_mileage'])
    print("error:", result['error'])
    print(result['planning'])
    print("." * 100)

rlefkee00@gmail.com
target: 30-32
actual: 32.0
error: 0.0
Planning(weekly_mileage_target=30-32 miles, long_run_distance=15.0, remaining_weekly_mileage=32 - 15 = 17 miles or 30 - 15 = 15 miles, remaining_weekly_mileage_planning=To distribute the remaining 17 miles over the 6 other days, let's plan as follows: Two days with easy 5 mile runs (17 - 10 = 7 miles remaining). One moderate run of 5 miles (7 - 5 = 2 miles remaining). One day with a short 2-mile recovery run (2 - 2 = 0 miles remaining). Plus 2 rest days.)
....................................................................................................
voynow99@gmail.com
target: 47-48
actual: 47.0
error: 0.0
Planning(weekly_mileage_target=47-48 miles, long_run_distance=17.0, remaining_weekly_mileage=47 - 17 = 30 miles, remaining_weekly_mileage_planning=To distribute 30 miles over the remaining 6 days: One session of 8 miles (30 - 8 = 22 miles remaining). One session of 7 miles (22 - 7 = 15 miles remaining). One session of 6 mi